In [3]:
# Importing necessary modules
from algosdk import account, mnemonic, transaction, constants
from algosdk.v2client import algod
import os
from dotenv import load_dotenv

### Create an Algorand Test account

In [4]:
# This function generates an algorand test account, returns a private and public key

testPrivateKey, testPublicKey = account.generate_account()

print("Public address: {}".format(testPublicKey))

Public address: ZXD5BWZ5TAOCMGWRSIB7JYMIC5CVGNH7XZCBJLMHW3URS5YJIUV2TPLGKI


### More Test Accounts

Connection to Algorand `dev` network comes with preconfigured accounts. We load them here.

In [5]:
load_dotenv()
preconPubKey01 = os.getenv('PreconfigPublicKey1')
preconPass01 = os.getenv('PreconfigPassphrase1')
preconPubKey02 = os.getenv('PreconfigPublicKey2')
preconPass02 = os.getenv('PreconfigPassphrase2')
preconPubKey03 = os.getenv('PreconfigPublicKey3')
preconPass03 = os.getenv('PreconfigPassphrase3')

In [6]:
print("Public address: {}".format(preconPubKey01))

Public address: None


In [7]:
# We use the passphrase to get the corresponding private key
preconPrivKey01 = mnemonic.to_private_key(preconPass01)
preconPrivKey02 = mnemonic.to_private_key(preconPass02)
preconPrivKey03 = mnemonic.to_private_key(preconPass03)

AttributeError: 'NoneType' object has no attribute 'lower'

## Connect to Client

In [8]:
algod_address = "http://localhost:4001" 
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa" 
algod_client = algod.AlgodClient(algod_token, algod_address)

In [9]:
# Check the account balance for two of the account we have
account_info = algod_client.account_info(testPublicKey)
print("Public address: {}".format(testPublicKey))
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

account_info = algod_client.account_info(preconPubKey01)
print("Public address: {}".format(preconPubKey01))
print("Account balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Public address: ZXD5BWZ5TAOCMGWRSIB7JYMIC5CVGNH7XZCBJLMHW3URS5YJIUV2TPLGKI
Account balance: 0 microAlgos



TypeError: can only concatenate str (not "NoneType") to str

## Build the transaction

In [9]:
# we are going to send 1 Algo from first preconfigured account to test account

sender_PubKey = preconPubKey01
sender_PrivKey = preconPrivKey01

receiver_PubKey = testPublicKey

In [10]:
account_info = algod_client.account_info(sender_PubKey)
print("Sender initial balance: {} microAlgos".format(account_info.get('amount')) + "\n")

account_info = algod_client.account_info(receiver_PubKey)
print("Receiver initial balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Sender initial balance: 4000000000000000 microAlgos

Receiver initial balance: 0 microAlgos



In [11]:
# Transaction Prameters 
params = algod_client.suggested_params()
params.flat_fee = True
params.fee = constants.MIN_TXN_FEE 
note = "Hello World".encode()
amount = 1000000

# Build Transaction
unsigned_txn = transaction.PaymentTxn(sender_PubKey, params, receiver_PubKey, amount, None, note)

In [12]:
# sign transaction using private key of the sender

signed_txn = unsigned_txn.sign(sender_PrivKey)

## Submit and Confirm the Transaction

At this moment the transaction has not yet been registered on Algorand (`dev`) network

In [13]:
#submit transaction

txid = algod_client.send_transaction(signed_txn)
print("Successfully sent transaction with txID: {}".format(txid))

Successfully sent transaction with txID: 4VKZ2DY3JKBKMFVD6QE3QF32KA3AU277NVRUTVBYIYYWPDEVTZOA


In [14]:
# wait for confirmation 
try:
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 4)  
except Exception as err:
    print(err)
    #return

print("Transaction information: {}".format(json.dumps(confirmed_txn, indent=4)))
print("Decoded note: {}".format(base64.b64decode(confirmed_txn["txn"]["txn"]["note"]).decode()))
print("Starting Account balance: {} microAlgos".format(account_info.get('amount')) )
print("Amount transfered: {} microAlgos".format(amount) )    
print("Fee: {} microAlgos".format(params.fee) ) 

Transaction information: {
    "confirmed-round": 1,
    "pool-error": "",
    "txn": {
        "sig": "C1STzJk96Fh/sHzPYbJa4gdGjUOYBdUC0LHHOHo2bqIfiASNqQIXeNVVxmtYeX4riG4oAM466h0FbzTWgdlXDA==",
        "txn": {
            "amt": 1000000,
            "fee": 1000,
            "gen": "sandnet-v1",
            "gh": "1Av0Ow3cvZTlmk8Nl+rYpdaYl8sFmSD4gie6oNNsKOA=",
            "lv": 1000,
            "note": "SGVsbG8gV29ybGQ=",
            "rcv": "IZ3BY7RKGHX6HL5HJKUICQPZQD4X5K3JADPQFJ5R7UAG4NB2BA2XQJP5ME",
            "snd": "BOBMBO72PB2AM4RBSHB7B3ZZML2AW4SDIUFKJ3YLCPSDKNO3FOCRMN7H2A",
            "type": "pay"
        }
    }
}
Decoded note: Hello World
Starting Account balance: 0 microAlgos
Amount transfered: 1000000 microAlgos
Fee: 1000 microAlgos


In [15]:
account_info = algod_client.account_info(sender_PubKey)
print("Sender final balance: {} microAlgos".format(account_info.get('amount')) + "\n")

account_info = algod_client.account_info(receiver_PubKey)
print("Receiver final balance: {} microAlgos".format(account_info.get('amount')) + "\n")

Sender final balance: 3999999998999000 microAlgos

Receiver final balance: 1000000 microAlgos



transaction has been successfuly registered on Algorand network!